In [1]:
!pip install tldextract
!pip install xgboost
!pip install pandas numpy beautifulsoup4 textblob scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install nltk
!pip install wordcloud
!pip install scipy
!pip install nltk
!pip install wordcloud


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import re
import urllib.parse
from collections import Counter
import tldextract
import matplotlib.pyplot as plt
import numpy as np


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score


def load_and_prepare_data():
    print("Basic Information About the Dataset")
    df = pd.read_csv('final_phishing_dataset.csv')
    
    X = df.drop('label', axis=1)
    y = df['label']
    
    print(f"Dataset shape: {X.shape}")
    print(f"Class distribution:\n{y.value_counts()}")
    
    return X, y

def identify_feature_types(X):
    # separate custom features from tf-idf features
    total_features = X.shape[1]
    tfidf_count = 5000
    
    if total_features > tfidf_count:
        custom_features = list(X.columns[:-tfidf_count])
        tfidf_features = list(X.columns[-tfidf_count:])
    else:
        custom_features = list(X.columns)
        tfidf_features = []
    
    print(f"Custom features: {len(custom_features)}")
    print(f"TF-IDF features: {len(tfidf_features)}")
    
    return custom_features, tfidf_features

def create_preprocessing_pipeline(custom_features, tfidf_features):
    # create models for different feature types
    transformers = []
    
    # Standardize the custom features because they are not on the same scale as the TF-IDF features
    if custom_features:
        transformers.append(('custom', StandardScaler(), custom_features))
    
    # Leave the tfidf features as they are
    if tfidf_features:
        transformers.append(('tfidf', 'passthrough', tfidf_features))
    
    preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')
    
    return preprocessor

def model_selection_pipeline(X, y, custom_features, tfidf_features):
    print("\n" + "="*60)
    print("MODEL SELECTION USING 5-FOLD CROSS-VALIDATION")
    print("="*60)
    
    # Define models to test with default parameters
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'XGBoost': XGBClassifier(n_estimators=100, random_state=42),
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'SVM': SVC(random_state=42, probability=True),
        'Naive Bayes': MultinomialNB()
    }
    
    preprocessor = create_preprocessing_pipeline(custom_features, tfidf_features)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    model_scores = {}
    best_model_name = None
    best_score = 0
    
    print("\nTesting models with default parameters...")
    print("-" * 60)
    
    for name, model in models.items():
        print(f"\nTesting {name}...")
        
        # Use different feature selection strategies for different feature types
        if custom_features and tfidf_features:
            # For mixed features, use a more robust approach without feature selection
            pipeline = Pipeline([
                ('preprocess', preprocessor),
                ('classifier', model)
            ])
        else:
            # For single feature type, use feature selection
            pipeline = Pipeline([
                ('preprocess', preprocessor),
                ('select', SelectKBest(f_classif, k=min(1000, X.shape[1]))),
                ('classifier', model)
            ])
        
        # Test multiple metrics
        metrics = {
            'f1_weighted': 'f1_weighted',
            'precision_weighted': 'precision_weighted', 
            'recall_weighted': 'recall_weighted',
            'accuracy': 'accuracy'
        }
        
        scores_dict = {}
        for metric_name, metric_scorer in metrics.items():
            try:
                scores = cross_val_score(pipeline, X, y, cv=cv, scoring=metric_scorer, n_jobs=-1)
                mean_score = scores.mean()
                std_score = scores.std()
                
                scores_dict[metric_name] = {
                    'mean': mean_score,
                    'std': std_score,
                    'scores': scores
                }
                
                print(f"{metric_name.capitalize()}: {mean_score:.4f} (+/- {std_score:.4f})")
                
            except Exception as e:
                print(f"Error with {metric_name}: {e}")
                scores_dict[metric_name] = {
                    'mean': np.nan,
                    'std': np.nan,
                    'scores': [np.nan] * 5
                }
        
        model_scores[name] = {
            'metrics': scores_dict,
            'model': model
        }
        
        # Use F1 score for model selection (as requested)
        f1_mean = scores_dict.get('f1_weighted', {}).get('mean', 0)
        if f1_mean > best_score and not np.isnan(f1_mean):
            best_score = f1_mean
            best_model_name = name
    
    return best_model_name, models[best_model_name], model_scores

def print_results(best_model_name, model_scores):
    print(f"\n{'='*60}")
    print("MODEL SELECTION RESULTS")
    print(f"{'='*60}")
    
    # Sort models by F1 performance
    sorted_models = sorted(
        model_scores.items(), 
        key=lambda x: x[1]['metrics'].get('f1_weighted', {}).get('mean', 0), 
        reverse=True
    )
    
    print("\nModel Rankings (by F1 Score):")
    print("-" * 60)
    for i, (name, scores) in enumerate(sorted_models, 1):
        f1_score = scores['metrics'].get('f1_weighted', {}).get('mean', np.nan)
        precision = scores['metrics'].get('precision_weighted', {}).get('mean', np.nan)
        recall = scores['metrics'].get('recall_weighted', {}).get('mean', np.nan)
        accuracy = scores['metrics'].get('accuracy', {}).get('mean', np.nan)
        
        print(f"{i}. {name}:")
        print(f"   F1: {f1_score:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | Accuracy: {accuracy:.4f}")
    
    print(f"\n{'='*60}")
    print(f"BEST MODEL: {best_model_name}")
    best_metrics = model_scores[best_model_name]['metrics']
    print(f"Best F1 Score: {best_metrics.get('f1_weighted', {}).get('mean', 'N/A')}")
    print(f"Best Precision: {best_metrics.get('precision_weighted', {}).get('mean', 'N/A')}")
    print(f"Best Recall: {best_metrics.get('recall_weighted', {}).get('mean', 'N/A')}")
    print(f"Best Accuracy: {best_metrics.get('accuracy', {}).get('mean', 'N/A')}")
    print(f"{'='*60}")


# MAIN SECTION WHERE EVERYTHING WILL BE EXECUTED

X, y = load_and_prepare_data()
custom_features, tfidf_features = identify_feature_types(X)
best_model_name, best_model, model_scores = model_selection_pipeline(
    X, y, custom_features, tfidf_features
)
print_results(best_model_name, model_scores)

print(f"\n{'='*60}")
print("Model Selection Completed")
print(f"{'='*60}")

Basic Information About the Dataset
Dataset shape: (200609, 2521)
Class distribution:
label
0.0    107534
1.0     93075
Name: count, dtype: int64
Custom features: 2521
TF-IDF features: 0

MODEL SELECTION USING 5-FOLD CROSS-VALIDATION

Testing models with default parameters...
------------------------------------------------------------

Testing Random Forest...
Error with f1_weighted: Unable to allocate 766. MiB for an array with shape (2502, 40122) and data type float64
Error with precision_weighted: Unable to allocate 766. MiB for an array with shape (2502, 40122) and data type float64


KeyboardInterrupt: 

In [ ]:
#Now we need to 